# Gogle Colab Set up

In [ ]:
!git clone https://github.com/Juanki0396/TFG

In [ ]:
import os
os.chdir("TFG")

In [ ]:
!pip install -r requirements.txt
!mkdir save_models
!bash dataset.sh

# Training a classifier

In this notebook we will run 3 training steps comparing the classifying capacity of a Resnet with 3 different scenarios:

1. Images with random labels
2. Images with and without added noise
3. Original images without noise and noise images treated with a CycleGan

In [3]:
import random

from torch.utils.data import DataLoader

from config import TRAIN_DATA_PATH, TEST_DATA_PATH, SEED
from src.data.dataset import ImageDataset, CycleGanDataset ,load_data, generate_noisy_dataset, generate_cycle_gan_dataset
from src.models.base_model import CycleGan, ImageClassifier
from src.models.trainning import ClassifierTrainer, CycleGanTrainer, train_classifiers, train_cyclegan
from src.utils import plot_image

train_data = load_data(TRAIN_DATA_PATH)
test_data = load_data(TEST_DATA_PATH)

random.seed(SEED)

## 1. Images with random labels

In [ ]:
epochs = 5
batch_size = 16
lr = 1e-3

model = ImageClassifier(lr=lr, threshold=0.5, device="cuda:0", name="Random_Labels_Clasiffier")

train_dataset = ImageDataset(train_data, random_labels=True)
test_dataset = ImageDataset(test_data, random_labels=True)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size//2, shuffle=False)

train_manager = ClassifierTrainer(model=model, save_dir="./save_models")
train_manager.set_dataloaders(train_dataloader, test_dataloader)
train_manager = train_classifiers(train_manager, epochs=5)
loss_fig = train_manager.plot_losses()


## 2. Images with and without added noise

In [ ]:
train_noisy_data = generate_noisy_dataset(train_data)
test_noisy_data = generate_noisy_dataset(test_data)

train_noisy_dataset = ImageDataset(train_noisy_data, random_labels=False)
test_noisy_dataset = ImageDataset(test_noisy_data, random_labels=False)

epochs = 7
batch_size = 16
lr = 1e-3

model = ImageClassifier(lr=lr, threshold=0.5, device="cuda:0", name="Noisy_Images_Clasiffier")
train_dataloader = DataLoader(train_noisy_dataset, batch_size=batch_size, shuffle=True )
test_dataloader = DataLoader(test_noisy_dataset, batch_size=1, shuffle=False)

train_manager = ClassifierTrainer(model, save_dir="./save_models")
train_manager.set_dataloaders(train_dataloader, test_dataloader)
train_manager = train_classifiers(train_manager, epochs=epochs)
loss_fig = train_manager.plot_losses()

## 3. Real and CycleGan Images

In [ ]:
cyclegan_train_data = generate_cycle_gan_dataset(train_data)
cyclegan_train_dataset = CycleGanDataset(cyclegan_train_data)
cyclegan_test_data = generate_cycle_gan_dataset(test_data)
cyclegan_test_dataset = CycleGanDataset(cyclegan_test_data)

epochs = 7
batch_size = 1
lr = 1e-3

model = CycleGan(device="cuda:0", lr=lr)
train_dataloader = DataLoader(cyclegan_train_dataset, batch_size=batch_size, shuffle=True)

train_manager = CycleGanTrainer(model, save_dir="./save_models")
train_manager.set_dataloaders(train_dataloader)
train_manager = train_cyclegan(train_manager, epochs=epochs)
fig = train_manager.plot_training_losses()